# Sentiment Analysis Model

### First step: Dataset loading and adjusting the chunk

In [2]:
# Import necessary libraries
import pandas as pd
import json
# Read JSON in chunks
chunk_size = 1000000   # Adjust the chunk size according to your system's memory
chunks = []
for chunk in pd.read_json('yelp_academic_dataset_review.json', lines=True, chunksize=chunk_size):
    chunks.append(chunk)
    break  # Remove this line to read the entire file in chunks
# Combine chunks into a DataFrame if needed
df = pd.concat(chunks, axis=0)
print(df.head())

                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   

   stars  useful  funny  cool  \
0      3       0      0     0   
1      5       1      0     1   
2      3       0      0     0   
3      5       1      0     1   
4      4       1      0     1   

                                                text                date  
0  If you decide to eat here, just be aware it is... 2018-07-07 22:09:11  
1  I've taken a lot of spin classes over the year... 2012-01-03 15:28:18  
2  Family diner. Had the buffet. Eclectic assortm... 2014-02-05 20:30:30  
3  Wow!  Yummy, different,  delicious.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   review_id    1000000 non-null  object        
 1   user_id      1000000 non-null  object        
 2   business_id  1000000 non-null  object        
 3   stars        1000000 non-null  int64         
 4   useful       1000000 non-null  int64         
 5   funny        1000000 non-null  int64         
 6   cool         1000000 non-null  int64         
 7   text         1000000 non-null  object        
 8   date         1000000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 68.7+ MB


In [5]:
df.columns.unique()

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

### Second step: It is time to get rid of unnecessary columns

In [6]:
df = df.drop(['review_id', 'user_id', 'business_id','funny','cool','date'], axis=1)

### 3. Function for labeling the sentiments and applying them

In [7]:
def label_sentiment(rating):
    if rating >= 4:
        return 2
    elif rating <= 2:
        return 0
    else:
        return 1

In [8]:
df['sentiment'] = df['stars'].apply(label_sentiment)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   stars      1000000 non-null  int64 
 1   useful     1000000 non-null  int64 
 2   text       1000000 non-null  object
 3   sentiment  1000000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 30.5+ MB


In [10]:
df['stars'].value_counts().sum()

1000000

In [12]:
len(df['text'].value_counts())

998357

In [13]:
len(df['sentiment'].value_counts())

3

### 4. Picking up the useful reviews

In [14]:
df = df[df['useful'] == 1].reset_index(drop=True)

In [15]:
df['stars'].value_counts().sum()

217272

In [16]:
len(df['text'].value_counts())

217168

In [17]:
len(df['sentiment'].value_counts())

3

In [18]:
sentiment_list = df['sentiment'].tolist()
 
print(sentiment_list)

[2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 0, 2, 1, 2, 2, 1, 2, 0, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 0, 1, 2, 2, 2, 2, 0, 2, 0, 1, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 1, 0, 0, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 1, 2, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 0, 1, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 1, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2, 

In [19]:
df['stars'].value_counts()

5    96614
4    49121
1    30505
3    23130
2    17902
Name: stars, dtype: int64

In [20]:
df.shape

(217272, 4)

In [21]:
df['sentiment'].value_counts()

2    145735
0     48407
1     23130
Name: sentiment, dtype: int64

### 5. Fixing and equalizing the sample sizes

In [22]:
import random

star_counts = df['stars'].value_counts()
desired_count = 17000
sampled_dfs = []


for star_rating in star_counts.index:
    subset_df = df[df['stars'] == star_rating]
    sampled_df = subset_df.sample(n=desired_count, random_state=42) 
    sampled_dfs.append(sampled_df)
    
df_equalized = pd.concat(sampled_dfs)


In [23]:
import random

sentiment_counts = df['sentiment'].value_counts()
desired_count_2 = 17000
sampled_dfs_2 = []


for sentiment_rating in sentiment_counts.index:
    subset_df_2 = df[df['sentiment'] == sentiment_rating]
    sampled_df_2 = subset_df_2.sample(n=desired_count_2, random_state=42) 
    sampled_dfs_2.append(sampled_df_2)
    
df_equalized = pd.concat(sampled_dfs_2)


In [24]:
df_equalized['stars'].value_counts()

3    17000
5    11244
1    10792
2     6208
4     5756
Name: stars, dtype: int64

In [25]:
df = df_equalized

In [26]:
df.isnull().sum()

stars        0
useful       0
text         0
sentiment    0
dtype: int64

### 6. Processing data - Tokenization

In [27]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    # Tokenization
    words = word_tokenize(text.lower())
    
    # Remove stop words and non-alphanumeric characters
    filtered_words = [word for word in words if word.isalnum() and word not in stop_words]
    
    # Stemming
    stemmed_words = [stemmer.stem(word) for word in filtered_words]
    
    return ' '.join(stemmed_words)

df['cleaned_text'] = df['text'].apply(preprocess_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Serhat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Serhat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51000 entries, 66337 to 101642
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   stars         51000 non-null  int64 
 1   useful        51000 non-null  int64 
 2   text          51000 non-null  object
 3   sentiment     51000 non-null  int64 
 4   cleaned_text  51000 non-null  object
dtypes: int64(3), object(2)
memory usage: 2.3+ MB


In [29]:
df.to_csv('clean_yelp_data.csv')

### 7. Splitting the data to test

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['cleaned_text'], df['sentiment'], test_size=0.2, random_state=42)

### 8. Vectorizing

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

### 9. Grid Searches and Classifition Reports

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {'n_estimators': [50, 100, 150],
              'max_depth': [5, 10, 20, 30]}

rf = RandomForestClassifier()

grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train_tfidf, y_train)


print("Best n_estimators for Random Forest:", grid_search.best_params_['n_estimators'])
print("Best max_depth for Random Forest:", grid_search.best_params_['max_depth'])
print("Best accuracy for Random Forest:", grid_search.best_score_)


Best n_estimators for Random Forest: 150
Best max_depth for Random Forest: 30
Best accuracy for Random Forest: 0.6992647058823529


In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('Random Forest', RandomForestClassifier(n_estimators=150, max_depth=30))])


clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)



print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.73      0.74      3471
           1       0.61      0.59      0.60      3332
           2       0.72      0.78      0.75      3397

    accuracy                           0.70     10200
   macro avg       0.70      0.70      0.70     10200
weighted avg       0.70      0.70      0.70     10200



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

param_grid = {'n_neighbors': list(range(1, 21))}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_train_tfidf, y_train)

print("Best K for KNN:", grid_search.best_params_['n_neighbors'])
print("Best accuracy for KNN:", grid_search.best_score_)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('KNN', KNeighborsClassifier(n_neighbors = 20))         
])


clf.fit(X_train, y_train)



y_pred = clf.predict(X_test)



print(classification_report(y_test, y_pred))

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB


param_grid = {'alpha': [0.01, 0.05, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]}


nb = MultinomialNB()


grid_search = GridSearchCV(nb, param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train_tfidf, y_train)


print("Best alpha for Multinomial NB:", grid_search.best_params_['alpha'])
print("Best accuracy for Multinomial NB:", grid_search.best_score_)


Best alpha for Multinomial NB: 0.5
Best accuracy for Multinomial NB: 0.6961274509803921


In [39]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB


clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('Multi NB', MultinomialNB(alpha=5))         
])


clf.fit(X_train, y_train)



y_pred = clf.predict(X_test)



print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.70      0.73      3471
           1       0.54      0.80      0.65      3332
           2       0.91      0.55      0.69      3397

    accuracy                           0.68     10200
   macro avg       0.74      0.68      0.69     10200
weighted avg       0.74      0.68      0.69     10200



### 10. Et Voila! We are ready!

The results proved to be much as expected. For opinionated texts, there is usually a 70% agreement between human raters. Thus, my model, which have accuracies around and above 70%, provide a very strong model for sentiment analysis. 


References: 

Queppelin Tech, NLP Report 
https://bit.ly/text_sentiment_analysis_based_on_NLP_works_for_businesses

Standard University, Jason Jang
https://cs229.stanford.edu/proj2011/Jong-%20PredictingRatingwithSentimentAnalysis.pdf#:~:text=The%20results%20proved%20to%20be%20much%20better%20than,provide%20a%20very%20strong%20model%20for%20sentiment%20analysis.

In [57]:
X_test

165709    wee book inn great altern book store sell new ...
4053      shop kingsway sinc kid chang lot sinc addit 80...
120745    sooooo cute realli alon give place 5 star love...
187909    quirki littl place decent food good servic tho...
134500    yesterday pm wait 30 minut order one thing two...
                                ...                        
163763    let say fan anyth ferguson resid support neigh...
127355    order grubhub driver call say outsid knew wron...
172628    okay limit review admittedli stop get mint cho...
40957     two thing like locat friendli servic big diner...
13141     nice bar catch friend qualiti beer beer asid p...
Name: cleaned_text, Length: 10200, dtype: object

### LET'S TRY THE MODEL:

0 : Negative Review


1 : Neutral Review


2 : Positive Review

#### TEST1:

In [58]:
new_reviews = ["this restaurant is amazing!", "the service was terrible.", "average experience."]


preprocessed_reviews = [preprocess_text(text) for text in new_reviews]
pd.Series(preprocessed_reviews)

0      restaur amaz
1    servic terribl
2     averag experi
dtype: object

In [61]:
predicted_sentiments = clf.predict(pd.Series(preprocessed_reviews))

In [62]:
print(predicted_sentiments)

[2 0 1]


#### TEST2:

In [66]:
newest_reviews = ["horrible service, never come back", "best bar in town", "it was alright but not recommend"]

preprocessed_reviews_2 = [preprocess_text(text) for text in newest_reviews]
pd.Series(preprocessed_reviews_2)

0    horribl servic never come back
1                     best bar town
2                 alright recommend
dtype: object

In [67]:
predicted_sentiments_2 = clf.predict(pd.Series(preprocessed_reviews_2))
print(predicted_sentiments_2)

[0 2 1]
